<a href="https://colab.research.google.com/github/MayerT1/ICEYE_DEV/blob/main/ICEYE_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://servir-mekong.github.io/hydra-floods/

https://www.iceye.com/lp/example-scan-dataset-panama-canal/thank-you-download-example-scan-dataset-panama-canal?submissionGuid=2622e80d-2f00-4544-9ca7-60cdea2d3483

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!ls

drive  sample_data


edge_otsu(img, band=None, region=None, scale=90, initial_threshold=0, thresh_no_data=None, invert=False, canny_threshold=0.05, canny_sigma=0, canny_lt=0.05, connected_pixels=200, edge_length=50, edge_buffer=100, max_buckets=255, min_bucket_width=0.001, max_raw=1000000.0, return_threshold=False)

In [ ]:
!pip install rasterio scikit-image matplotlib

import rasterio
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# === File paths ===
input_path = '/content/drive/MyDrive/ICEYE_Data/ICEYE_Export.tif'
output_path = '/content/drive/MyDrive/ICEYE_Data/ICEYE_EdgeOtsu_Classification.tif'

# === Load TIFF ===
with rasterio.open(input_path) as src:
    band1 = src.read(1)
    profile = src.profile

# === Handle nodata ===
nodata = profile.get('nodata', None)
if nodata is not None:
    band1 = np.where(band1 == nodata, np.nan, band1)

# === Apply Otsu thresholding ===
band1_valid = band1[~np.isnan(band1)]
threshold = threshold_otsu(band1_valid)
print(f"Otsu Threshold: {threshold}")

# Create binary mask
binary_mask = (band1 > threshold).astype(np.uint8)

# === Save output TIFF ===
output_profile = profile.copy()
output_profile.update({
    'dtype': 'uint8',
    'count': 1,
    'nodata': 0
})

with rasterio.open(output_path, 'w', **output_profile) as dst:
    dst.write(binary_mask, 1)

print(f"Saved classified image to: {output_path}")

# === Plot original and result ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

ax1.set_title("Original Band")
im1 = ax1.imshow(band1, cmap='gray')
plt.colorbar(im1, ax=ax1)

ax2.set_title("Otsu Thresholded Mask")
im2 = ax2.imshow(binary_mask, cmap='gray')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()


Otsu Threshold: 84
Saved classified image to: /content/drive/MyDrive/ICEYE_Data/ICEYE_EdgeOtsu_Classification.tif
